# 03 - Ingredient Data Cleaning
## Paula's Choice Ingredient Dictionary Analysis

This notebook cleans and explores the scraped ingredient data from Paula's Choice.

### Objectives:
1. Load both CSV files
2. Explore data structure and quality
3. Identify and remove duplicates
4. Create a clean, unified dataset
5. Perform exploratory data analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Seaborn styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data

In [ ]:
# Load both CSV files
df1 = pd.read_csv('/mnt/user-data/uploads/paulas_choice_perfect_glance__1.csv')
df2 = pd.read_csv('/mnt/user-data/uploads/paulas_choice_perfect_glance_2.csv')

print(f"File 1 shape: {df1.shape}")
print(f"File 2 shape: {df2.shape}")
print(f"\nTotal rows before merge: {df1.shape[0] + df2.shape[0]}")

## 2. Initial Data Exploration

In [ ]:
# Display column names
print("Column names:")
print(df1.columns.tolist())

In [ ]:
# Display first few rows from each file
print("First 3 rows from File 1:")
display(df1.head(3))

print("\nFirst 3 rows from File 2:")
display(df2.head(3))

In [ ]:
# Check data types and missing values
print("File 1 Info:")
df1.info()

print("\n" + "="*50 + "\n")

print("File 2 Info:")
df2.info()

## 3. Check for Duplicates

In [ ]:
# Check for duplicate ingredient names within each file
# Note: We only check ingredient_name for duplicates, not rating/benefits/categories
# (those columns will naturally have duplicates across different ingredients)
print("Duplicate INGREDIENT NAMES in File 1:")
dup1 = df1['ingredient_name'].duplicated().sum()
print(f"  Count: {dup1}")

print("\nDuplicate INGREDIENT NAMES in File 2:")
dup2 = df2['ingredient_name'].duplicated().sum()
print(f"  Count: {dup2}")

# Show some duplicate examples if they exist
if dup1 > 0:
    print("\nDuplicate ingredient name examples in File 1:")
    dup_names = df1[df1['ingredient_name'].duplicated(keep=False)]['ingredient_name'].unique()[:5]
    for name in dup_names:
        print(f"  - {name}")
        display(df1[df1['ingredient_name'] == name][['ingredient_name', 'rating', 'categories']])

if dup2 > 0:
    print("\nDuplicate ingredient name examples in File 2:")
    dup_names = df2[df2['ingredient_name'].duplicated(keep=False)]['ingredient_name'].unique()[:5]
    for name in dup_names:
        print(f"  - {name}")
        display(df2[df2['ingredient_name'] == name][['ingredient_name', 'rating', 'categories']])

In [ ]:
# Check for overlap between files
overlap = set(df1['ingredient_name']) & set(df2['ingredient_name'])
print(f"Number of overlapping ingredients between files: {len(overlap)}")

if len(overlap) > 0:
    print("\nFirst 10 overlapping ingredients:")
    print(sorted(list(overlap))[:10])

## 4. Merge and Deduplicate

In [ ]:
# Concatenate both dataframes
df_combined = pd.concat([df1, df2], ignore_index=True)

print(f"Combined shape before deduplication: {df_combined.shape}")
print(f"Unique ingredients before deduplication: {df_combined['ingredient_name'].nunique()}")

In [ ]:
# Remove duplicates based on ingredient_name (keep first occurrence)
df_clean = df_combined.drop_duplicates(subset='ingredient_name', keep='first')

print(f"\nCleaned shape after deduplication: {df_clean.shape}")
print(f"Unique ingredients after deduplication: {df_clean['ingredient_name'].nunique()}")
print(f"\nRows removed: {df_combined.shape[0] - df_clean.shape[0]}")

In [ ]:
# Reset index
df_clean = df_clean.reset_index(drop=True)

# Display sample
print("Sample of cleaned data:")
display(df_clean.head(10))

## 5. Data Quality Analysis

In [ ]:
# Check for missing values
print("Missing values per column:")
missing = df_clean.isnull().sum()
missing_pct = (missing / len(df_clean)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
display(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))

In [ ]:
# Check for empty strings
print("Empty string counts per column:")
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        empty_count = (df_clean[col] == '').sum()
        if empty_count > 0:
            print(f"  {col}: {empty_count} ({empty_count/len(df_clean)*100:.2f}%)")

## 6. Exploratory Data Analysis

In [ ]:
# Extract rating from rating column (remove 'Rating: ' prefix)
df_clean['rating_clean'] = df_clean['rating'].str.replace('Rating: ', '', regex=False)

# Rating distribution
print("Rating Distribution:")
rating_counts = df_clean['rating_clean'].value_counts()
print(rating_counts)
print(f"\nSample ratings to verify cleaning:")
print(df_clean[['ingredient_name', 'rating', 'rating_clean']].head(10))

# Visualization
plt.figure(figsize=(10, 6))
rating_order = ['Best', 'Good', 'Average', 'Bad', 'Worst']
rating_counts = rating_counts.reindex([r for r in rating_order if r in rating_counts.index])
rating_counts.plot(kind='bar', color=['#2ecc71', '#3498db', '#f39c12', '#e67e22', '#e74c3c'])
plt.title('Distribution of Ingredient Ratings', fontsize=14, fontweight='bold')
plt.xlabel('Rating', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Benefits analysis - convert to list format
def extract_benefits(benefits_str):
    """Extract benefits from string and return as list"""
    if pd.isna(benefits_str) or benefits_str == '':
        return []
    if isinstance(benefits_str, str) and benefits_str.startswith('Benefits:'):
        benefits = benefits_str.replace('Benefits: ', '').split(', ')
        return [b.strip() for b in benefits if b.strip()]
    return []

df_clean['benefits_list'] = df_clean['benefits'].apply(extract_benefits)

has_benefits = df_clean['benefits_list'].apply(len) > 0
print("Ingredients with benefits:")
print(f"  Count: {has_benefits.sum()}")
print(f"  Percentage: {has_benefits.sum()/len(df_clean)*100:.2f}%")

# Extract unique benefits
all_benefits = [benefit for benefits_list in df_clean['benefits_list'] for benefit in benefits_list]

if all_benefits:
    benefits_series = pd.Series(all_benefits)
    print("\nMost common benefits:")
    print(benefits_series.value_counts().head(10))
    
print("\nSample benefits data:")
display(df_clean[has_benefits][['ingredient_name', 'benefits', 'benefits_list']].head(5))

In [ ]:
# Categories analysis - convert to list format
def extract_categories(categories_str):
    """Extract categories from string and return as list"""
    if pd.isna(categories_str) or categories_str == '':
        return []
    if isinstance(categories_str, str) and categories_str.startswith('Categories:'):
        categories = categories_str.replace('Categories: ', '').split(', ')
        return [c.strip() for c in categories if c.strip()]
    return []

df_clean['categories_list'] = df_clean['categories'].apply(extract_categories)

has_categories = df_clean['categories_list'].apply(len) > 0
print("Ingredients with categories:")
print(f"  Count: {has_categories.sum()}")
print(f"  Percentage: {has_categories.sum()/len(df_clean)*100:.2f}%")

# Extract unique categories
all_categories = [category for categories_list in df_clean['categories_list'] for category in categories_list]

if all_categories:
    categories_series = pd.Series(all_categories)
    print("\nMost common categories:")
    top_categories = categories_series.value_counts().head(15)
    print(top_categories)
    
    # Visualization
    plt.figure(figsize=(12, 8))
    top_categories.plot(kind='barh', color='steelblue')
    plt.title('Top 15 Ingredient Categories', fontsize=14, fontweight='bold')
    plt.xlabel('Count', fontsize=12)
    plt.ylabel('Category', fontsize=12)
    plt.tight_layout()
    plt.show()
    
print("\nSample categories data:")
display(df_clean[has_categories][['ingredient_name', 'categories', 'categories_list']].head(5))

In [ ]:
# INFO_GLANCE analysis
print("Ingredients with detailed info:")
has_info = df_clean['INFO_GLANCE'].notna() & (df_clean['INFO_GLANCE'] != '')
print(f"  Count: {has_info.sum()}")
print(f"  Percentage: {has_info.sum()/len(df_clean)*100:.2f}%")

# Check for sentence separation issues
print("\n" + "="*80)
print("CHECKING INFO_GLANCE FORMATTING ISSUES")
print("="*80)

# Sample ingredients with info to check formatting
sample_info = df_clean[has_info][['ingredient_name', 'INFO_GLANCE']].head(10)

print("\nSample INFO_GLANCE entries (checking for sentence gluing):")
for idx, row in sample_info.iterrows():
    print(f"\n{'='*80}")
    print(f"Ingredient: {row['ingredient_name']}")
    print(f"{'='*80}")
    info_text = row['INFO_GLANCE']
    # Check if there are lowercase letters immediately after uppercase (sign of glued sentences)
    import re
    potential_glue = re.findall(r'[a-z][A-Z]', info_text)
    if potential_glue:
        print(f"⚠️  POTENTIAL GLUED SENTENCES DETECTED: {potential_glue}")
    print(f"Text: {info_text[:200]}..." if len(info_text) > 200 else f"Text: {info_text}")
    print()

# Count how many have potential issues
def check_glued_sentences(text):
    if pd.isna(text) or text == '':
        return False
    import re
    return bool(re.search(r'[a-z][A-Z]', text))

glued_count = df_clean[has_info]['INFO_GLANCE'].apply(check_glued_sentences).sum()
print(f"\n⚠️  Entries with potential glued sentences: {glued_count} out of {has_info.sum()} ({glued_count/has_info.sum()*100:.1f}%)")

## 7. Fix INFO_GLANCE Sentence Separation

The INFO_GLANCE data contains bullet points that were scraped without separators.
Let's convert them into a proper list structure (like benefits and categories).

In [ ]:
# Split glued sentences into list of bullet points
def extract_info_bullets(text):
    """Split INFO_GLANCE text into list of bullet points"""
    if pd.isna(text) or text == '':
        return []
    
    import re
    # Split on pattern: lowercase followed by uppercase (start of new sentence)
    # This captures bullet point boundaries
    sentences = re.split(r'(?<=[a-z])(?=[A-Z])', text)
    
    # Clean up each sentence
    bullets = [s.strip() for s in sentences if s.strip()]
    
    return bullets

df_clean['info_bullets'] = df_clean['INFO_GLANCE'].apply(extract_info_bullets)

# Show before/after examples
print("Before and After - Converting to bullet point lists:")
print("="*80)
sample_info = df_clean[df_clean['INFO_GLANCE'].notna() & (df_clean['INFO_GLANCE'] != '')].head(5)

for idx, row in sample_info.iterrows():
    print(f"\n{'='*80}")
    print(f"Ingredient: {row['ingredient_name']}")
    print(f"{'='*80}")
    print(f"\nBEFORE (glued text):")
    print(f"  {row['INFO_GLANCE'][:200]}..." if len(row['INFO_GLANCE']) > 200 else f"  {row['INFO_GLANCE']}")
    print(f"\nAFTER (bullet list):")
    for i, bullet in enumerate(row['info_bullets'], 1):
        print(f"  {i}. {bullet}")

# Statistics
has_info = df_clean['info_bullets'].apply(len) > 0
avg_bullets = df_clean[has_info]['info_bullets'].apply(len).mean()
print(f"\n{'='*80}")
print(f"📊 INFO_GLANCE Statistics:")
print(f"  Ingredients with info: {has_info.sum()} ({has_info.sum()/len(df_clean)*100:.1f}%)")
print(f"  Average bullet points per ingredient: {avg_bullets:.1f}")
print(f"  Data structure: List of strings (like benefits/categories)")
print(f"="*80)

## 8. Clean Column Selection

Let's select only the relevant columns for the SkinGen project.

In [ ]:
# Select relevant columns including the cleaned list versions
df_final = df_clean[[
    'ingredient_name',
    'rating_clean',
    'benefits_list',
    'categories_list',
    'info_bullets'
]].copy()

# Rename for clarity
df_final.columns = ['ingredient_name', 'rating', 'benefits', 'categories', 'info']

print(f"Final dataset shape: {df_final.shape}")
print(f"\nColumn data types:")
print(df_final.dtypes)
print(f"\nSample of final data:")
display(df_final.head())

print("\n📋 All columns are now in LIST format:")
sample_row = df_final[df_final['info'].apply(len) > 0].iloc[0]
print(f"\nExample - {sample_row['ingredient_name']}:")
print(f"  Rating: {sample_row['rating']}")
print(f"  Benefits: {sample_row['benefits']}")
print(f"  Categories: {sample_row['categories']}")
print(f"  Info bullets:")
for i, bullet in enumerate(sample_row['info'], 1):
    print(f"    {i}. {bullet}")

## 9. Data Validation

In [ ]:
# Final checks
print("Final Data Quality Checks:")
print(f"1. Total ingredients: {len(df_final)}")
print(f"2. Unique ingredients: {df_final['ingredient_name'].nunique()}")
print(f"3. Duplicate ingredient names: {df_final['ingredient_name'].duplicated().sum()}")
print(f"4. Missing ingredient names: {df_final['ingredient_name'].isnull().sum()}")
print(f"5. Missing ratings: {df_final['rating'].isnull().sum()}")

# Check rating values
print("\n6. Valid rating values:")
print(df_final['rating'].unique())

# Check list columns
print("\n7. Benefits list format check:")
print(f"   Type: {type(df_final['benefits'].iloc[0])}")
sample_with_benefits = df_final[df_final['benefits'].apply(len) > 0].iloc[0]
print(f"   Sample: {sample_with_benefits['ingredient_name']} -> {sample_with_benefits['benefits']}")

print("\n8. Categories list format check:")
print(f"   Type: {type(df_final['categories'].iloc[0])}")
sample_with_cats = df_final[df_final['categories'].apply(len) > 0].iloc[0]
print(f"   Sample: {sample_with_cats['ingredient_name']} -> {sample_with_cats['categories']}")

print("\n9. Info bullets list format check:")
print(f"   Type: {type(df_final['info'].iloc[0])}")
sample_with_info = df_final[df_final['info'].apply(len) > 0].iloc[0]
print(f"   Sample: {sample_with_info['ingredient_name']}")
print(f"   Bullets: {sample_with_info['info'][:3]}...")  # Show first 3 bullets

## 10. Save Cleaned Data

In [ ]:
# Save to Parquet - preserves list structure natively!
parquet_path = '/mnt/user-data/outputs/ingredients_cleaned.parquet'
df_final.to_parquet(parquet_path, index=False, engine='pyarrow')
print(f"✅ Cleaned data saved to: {parquet_path}")
print("   Parquet format preserves list structure natively!")
print("   Load with: pd.read_parquet('ingredients_cleaned.parquet')")

# Also save as CSV for compatibility (with JSON-encoded lists)
import json
df_to_save = df_final.copy()
df_to_save['benefits'] = df_to_save['benefits'].apply(json.dumps)
df_to_save['categories'] = df_to_save['categories'].apply(json.dumps)
df_to_save['info'] = df_to_save['info'].apply(json.dumps)

csv_path = '/mnt/user-data/outputs/ingredients_cleaned.csv'
df_to_save.to_csv(csv_path, index=False)
print(f"\n✅ Also saved as CSV (for compatibility): {csv_path}")
print("   Note: All list columns are JSON strings in CSV")

# Compare file sizes
import os
parquet_size = os.path.getsize(parquet_path) / 1024  # KB
csv_size = os.path.getsize(csv_path) / 1024  # KB
print(f"\n📊 File size comparison:")
print(f"   Parquet: {parquet_size:.2f} KB")
print(f"   CSV: {csv_size:.2f} KB")
print(f"   Compression ratio: {csv_size/parquet_size:.2f}x")

# Display summary statistics
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Original files combined: {df1.shape[0] + df2.shape[0]} rows")
print(f"Duplicates removed: {(df1.shape[0] + df2.shape[0]) - len(df_final)}")
print(f"Final clean dataset: {len(df_final)} unique ingredients")
print(f"\nData structure (ALL as lists):")
print(f"  - ingredient_name: string")
print(f"  - rating: string (Best/Good/Average/Bad/Worst)")
print(f"  - benefits: list of strings")
print(f"  - categories: list of strings")
print(f"  - info: list of strings (bullet points)")
print("\n💡 Recommendation: Use Parquet for your ML pipeline!")
print("="*60)

## 11. Sample Analysis: Ingredient Quality by Category

Let's analyze which categories tend to have better or worse ratings.

In [ ]:
# Create a rating score for analysis
rating_score = {'Best': 5, 'Good': 4, 'Average': 3, 'Bad': 2, 'Worst': 1}
df_final['rating_score'] = df_final['rating'].map(rating_score)

# Explode categories to have one row per category (categories are already lists)
df_exploded = df_final.explode('categories')
df_exploded = df_exploded[df_exploded['categories'] != '']
df_exploded = df_exploded[df_exploded['categories'].notna()]

# Calculate average rating per category
category_ratings = df_exploded.groupby('categories')['rating_score'].agg(['mean', 'count']).reset_index()
category_ratings.columns = ['Category', 'Avg_Rating', 'Count']
category_ratings = category_ratings[category_ratings['Count'] >= 10]  # Filter for categories with at least 10 ingredients
category_ratings = category_ratings.sort_values('Avg_Rating', ascending=False)

print("Top 10 highest-rated categories (min 10 ingredients):")
display(category_ratings.head(10))

print("\nTop 10 lowest-rated categories (min 10 ingredients):")
display(category_ratings.tail(10))

In [ ]:
# Visualize top and bottom categories
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Top categories
top_10 = category_ratings.head(10)
ax1.barh(top_10['Category'], top_10['Avg_Rating'], color='green', alpha=0.7)
ax1.set_xlabel('Average Rating Score', fontsize=12)
ax1.set_title('Top 10 Highest-Rated Categories', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 5)
ax1.invert_yaxis()

# Bottom categories
bottom_10 = category_ratings.tail(10)
ax2.barh(bottom_10['Category'], bottom_10['Avg_Rating'], color='red', alpha=0.7)
ax2.set_xlabel('Average Rating Score', fontsize=12)
ax2.set_title('Top 10 Lowest-Rated Categories', fontsize=14, fontweight='bold')
ax2.set_xlim(0, 5)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

## Conclusion

The ingredient data has been successfully cleaned and prepared for the SkinGen project. Key findings:

- Started with 2 files containing potentially overlapping data
- Removed duplicates to create a clean dataset
- Analyzed ingredient ratings, benefits, and categories
- Identified high-quality and potentially problematic ingredient categories

**Next Steps:**
1. Use this cleaned ingredient database for product analysis
2. Develop the ingredient intelligence layer for SkinGen
3. Integrate with product recommendations system